In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision.transforms import ToTensor, Lambda

from PIL import Image, ImageDraw
import pandas as pd

f"/nsls2/users/maire1/sim_data/sim{i}.csv"

In [2]:
def set_rand_seed(num):
    '''
    Creates random seed so that randomness is repeatable. Initialized first to set all randomness
    '''
    np.random.seed(num)
    
def set_k():
    """
    Uses random seed to randomly set k - changes size of the speckles
    """
    return(np.random.randint(5,100))

def make_mesh(window_size):
    """
    Creates 2D meshgrid for calculating all the functions.
    """
    x = np.arange(window_size)
    y = np.arange(window_size)
    xx, yy = np.meshgrid(x,y)
    return xx,yy

def object_shape(x, y, x_center, y_center, sigma):
    """
    Defines the shape of the average intensity, can be changed for any other form.
    """
    return np.exp(-((x-x_center)**2 + (y-y_center)**2)/2/sigma)
           
def create_object(window_size = 1024, sigma = 5e4):
    """
    Calculates the average intensity on the defined mesh (i.e. on the CDD detector).
    """
    xx, yy = make_mesh(window_size)
    z = object_shape(xx, yy, 350,350, sigma)
    return z

def circle(r):
    return r<1

def bandpass(p,q,r0, n):
    """
    Defines the functional form of the lens. Lens is needed to make a point source.
    """
    return circle(np.sqrt((p-n/2)**2+(q-n/2)**2)/r0)

def make_bandpass(window_size, r0):
    """
    Calculates the lens band pass form on the defined mesh grid.
    """
    xx, yy = make_mesh(window_size)
    band_pass = bandpass(xx, yy, r0, window_size)
    return band_pass

In [3]:
def make_speckles(n, r0):
    """
    Creates speckles
    """
    # random phasors
    rand_field = np.random.rand(n,n) * 1j
    rand_field = np.exp(2*np.pi*rand_field)

    # object intensity in the absence of the speckle        
    object_intensity = create_object(window_size = n)

    # multiply the amplitude distribution by random phasors
    scattered_field = np.sqrt(object_intensity)*rand_field

    # circular function of the lens
    band_pass = make_bandpass(n, r0)

    # field transmitted by the lens
    pupil_field = band_pass*np.fft.fft2(scattered_field)


    image_field = np.fft.ifft2(pupil_field)
    image_intensity = np.abs(image_field)**2
    
    return(image_intensity)


def make_beam_streaks(window_size):
    """
    Creates a random number of triangles to simulate beam streaks. Coordinates are randomly generated around the window.
    Returns: np.array. For masking purposes, value is set to 1
    """
    
    blank = Image.new('L', (window_size,window_size),0)
    #Create random number of beam streaks between 1 and 3
    num_beam_streaks = np.random.randint(1,3)
    
    for i in range(num_beam_streaks):
        #Generating three random vertices
        rand_points = window_size * (np.random.rand(2,2))
        points = np.append(rand_points, [[window_size*np.random.random_sample(), np.sqrt(rand_points[0,1]**2 + rand_points[1,1]**2)]], axis=0)
        
        #Changing format of data points. Yes, there is likely an easier method to do this rather than hardcoding
        a = [(points[0,0],points[0,1]), (points[1,0],points[1,1]), (points[2,0],points[2,1])]
        ImageDraw.Draw(blank).polygon(a, outline=1, fill = 1)
    
    return(np.asarray(blank))

def make_beam_stop(n):
    """
    Creates coordinates and image for beam stop
    Returns: Beam stop as array. For masking purposes, value is set to 2
    """
    rect_coord = [(1/8*n, 1/8*n), (1/8*n, 1/4*n), (1/4*n, 1/4*n), (1/4*n, 1/8*n)]
    r = Image.new('L', (n,n), 0)
    ImageDraw.Draw(r).polygon(rect_coord, outline=2, fill=2)
    
    r = np.asarray(r)
    r = np.where(r==2, np.nan, r) 

    return(r)

def add_noise(window_size):
    """
    Adds random noise to image.
    """
    random_array = np.random.rand(window_size,window_size)
    return(random_array)


def save(sample,i, sample_name, sample_address):
    """
    Saves file in sim_data folder with iterating number to correspond to number of samples desired as pytorch tensors
    """
    input_tensor = torch.Tensor(sample['input'])
    target_tensor = torch.Tensor(sample['target'])

    torch.save({"input": input_tensor, "target": target_tensor}, f"/nsls2/users/maire1/sim_data/sim{i}.pt")
    
    #pd.Series(sample).to_frame().to_csv(f"/Users/morganaire/Documents/BNL/speckle_sim/sim_data/sim{i}.csv")
    #np.save(f'sim{i}', sample)
    sample_name.append(f"sim{i}.pt")
    sample_address.append(f"/nsls2/users/maire1/sim_data/sim{i}.pt")
    
    return(sample_name, sample_address)

def save_total_data(name, address):
    """
    Saving titles and sample addresses into a separate csv file for use in the neural network.
    
    TODO: Save as a torch tensor in the future. 
    """
    
    d = {'sample': name, 'address': address}
    df = pd.DataFrame(data=d)
    
    #torch.save(df)
    df.to_csv('/nsls2/users/maire1/sim_data/sim_address.csv', index = False)
    print("All samples completed. Data saved.")
    
def rescale(intensity, noise):
    '''
    This function rescales the intensity so that the speckles are always visible above the noise.
    This should also add some complexity to the simulation because speckle data is closer in intensity to beam streaks (beam streaks are around 1, speckles are now set to be > noise (~1))
    Returns: new intensity function
    '''
    while intensity.max() < noise:
        intensity = intensity * 5
    return intensity
    
def create_a_sample(rand_seed_num, n):
    """
    Does everything necessary to create both a full sample along with masks. 
    Reutrns: Dictionary with imput and target to be saved.
    """
    #Set initial params
    set_rand_seed(rand_seed_num)
    
    k = set_k()
    r0 = n/k
    
    #Creating all pieces of image
    image_intensity = make_speckles(n, r0)
    size = image_intensity.shape
    mask = np.zeros(size)
    Polygon = make_beam_streaks(n)
    beam_stop = make_beam_stop(n)
    
    img_intensity_rescaled = rescale(image_intensity,add_noise(n).max()) #rescale intensity value so that it is always visible and greater than the noise
    
    # Creating total image and setting beam stop values to zero
    new_image = img_intensity_rescaled + Polygon + beam_stop + add_noise(n)
    new_image = np.nan_to_num(new_image, nan=0)
    
    # Creating mask and setting beam stop values to zero
    mask = Polygon + mask + beam_stop
    mask = np.nan_to_num(mask, nan=0)
    print(mask.max(), new_image.max())

    #plt.imshow(new_image)
    #plt.show()
    sample_dict = {'input':new_image, 'target': mask}
    
    return (sample_dict)

In [4]:
'''MAIN'''
samples = 1000
window_size = 256

num_samples = np.arange(0,samples)
sn = []
sa = []

for j in (num_samples):
    x = create_a_sample(j, window_size)
    sn, sa = save(x,j, sn, sa)
save_total_data(sn, sa)

1.0 3.3494677522714595
1.0 4.127913854423167
1.0 3.805788363152658
1.0 2.5827455373832713
1.0 3.441524259170177
1.0 3.213817682751558
1.0 3.3336341301774053
1.0 3.1213174808099002
1.0 2.4367138420582117
1.0 2.4059726053300072
1.0 2.856969471540875
1.0 2.9359401444802664
1.0 5.73195361901619
1.0 3.3459213509295873
1.0 2.9184915413860035
1.0 5.42963997527231
1.0 4.2574551697584315
1.0 5.065463510877124
1.0 3.835914665929833
1.0 3.821240272461538
1.0 3.635947505854752
1.0 5.117031379759611
1.0 2.917217143666618
1.0 3.090223110022624
1.0 4.06432027447193
1.0 2.9499776096720094
1.0 3.171605509522418
1.0 4.414706448247026
1.0 3.4507731844060086
1.0 4.008143112981048
1.0 5.215372865693217
1.0 3.946107333336785
1.0 3.1936258475212234
1.0 4.038287773269023
1.0 2.6050546930696528
1.0 5.087455677850751
1.0 5.226277499719103
1.0 4.801053538495513
1.0 5.949229868109726
1.0 3.3549712139354866
1.0 3.7241378943007817
1.0 5.488267033572759
1.0 3.6987555118197157
1.0 2.7564370263035127
1.0 3.85660173168

1.0 4.740953825197343
1.0 2.872109834625645
1.0 2.5874657052323293
1.0 3.0725015689578488
1.0 2.423526217270849
1.0 5.686755991058839
1.0 5.124063390265491
1.0 2.8888533682707767
1.0 4.6301605351562225
1.0 2.525984835581091
1.0 3.480980514020497
1.0 4.629760898725639
1.0 2.9409765488684814
1.0 2.1919437839254248
1.0 3.697048616431089
1.0 2.75830298232145
1.0 3.6272000519838143
1.0 2.8717911572423147
1.0 2.9018527310845768
1.0 2.8762083178522495
1.0 2.911150372884632
1.0 4.573941439403033
1.0 4.872710335516943
1.0 2.3037167040122117
1.0 2.706536648662833
1.0 4.689012258740035
1.0 3.7157655369246188
1.0 4.069445264286587
1.0 3.1966860232707024
1.0 4.450962998394775
1.0 3.22445967150111
1.0 3.014757580567908
1.0 4.713993152920873
1.0 2.469162763103322
1.0 3.311383090316934
1.0 3.826102600175041
1.0 3.115152712968949
1.0 3.152391916291314
1.0 3.9350899541653876
1.0 5.781944922290682
1.0 4.274577038163623
1.0 2.862838263221197
1.0 3.5216841251964572
1.0 4.844895213253858
1.0 4.7817877814399

1.0 3.312721990696817
1.0 3.35020555521256
1.0 3.433587173056264
1.0 2.8988071949090757
1.0 2.7963537630021245
1.0 4.226938418522529
1.0 3.5996147150177
1.0 4.992539437664804
1.0 3.576694417763979
1.0 4.739852144727868
1.0 2.5544141813717314
1.0 3.4683133243160373
1.0 3.854393619899815
1.0 5.2636431627871065
1.0 3.126740355311645
1.0 2.37550682539995
1.0 4.582954629064352
1.0 4.490096413260469
1.0 4.029538073142948
1.0 5.713748605632182
1.0 2.9914236605113333
1.0 3.544331187074586
1.0 2.4369615768448196
1.0 2.2511826113522595
1.0 2.4736125722781237
1.0 2.8332345748828764
1.0 2.4805980277413235
1.0 2.57638416031532
1.0 3.210223517004124
1.0 3.2114635971172767
1.0 5.456760678541241
1.0 5.736652544247803
1.0 2.755284656491781
1.0 4.443291851784781
1.0 4.404888981121064
1.0 3.0740748736156114
1.0 2.951321764218809
1.0 3.580001228430733
1.0 4.338280045523077
1.0 4.6637355867941634
1.0 3.5823394506817436
1.0 2.6192855323528814
1.0 3.6632636063811086
1.0 3.2343690783429393
1.0 3.1427745189098